In [22]:
import sys, os
import numpy as np
from pyhdf.SD import SD
import pandas as pd
from datetime import datetime, time, timedelta
import re
import time
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import pearsonr

from functions import *
# Path and Variable declarations
# External harddrive path
file_ext= "D:\\NASAdataSocorroHueco"
file_path_plots=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\Plots"
# Internal testing folder
#file_ext=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\AIRS"
#sort file list
file_list=sorted(os.listdir(file_ext)[1:])
#test_list=file_list[:1]


# File names for TECQ PM2.5 stations in West Texas
file_name_cams49=r'CAMS_49_SocorroHueco.xlsx'
file_name_cams1028=r'CAMS_1028_Lubbock.xlsx'

# Coordinates for TECQ stations
CAMS49 = [31.6676,-106.288]
CAMS1028 = [33.5856, -101.78]


In [88]:
# Read both TECQ files into dataframes
tecq_cams49 = read_xlsx_tecq(file_name_cams49)
tecq_cams1028= read_xlsx_tecq(file_name_cams1028)
#tecq_cams49.info()
tecq_cams49
tecq_cams1028

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,4.0,5.0,2.0,3.0,4.0,3.0,3.0,7.0,2.0,3.0,...,0.0,6.0,4.0,3.0,3.0,3.0,1.0,2.0,2.0,4.0
2019-01-02,-1.0,3.0,2.0,2.0,4.0,2.0,4.0,6.0,1.0,7.0,...,3.0,8.0,4.0,8.0,12.0,10.0,8.0,7.0,7.0,10.0
2019-01-03,10.0,16.0,12.0,9.0,7.0,6.0,6.0,6.0,2.0,6.0,...,0.0,3.0,2.0,4.0,3.0,5.0,4.0,1.0,2.0,2.0
2019-01-04,-2.0,2.0,0.0,-3.0,-1.0,-3.0,1.0,0.0,-4.0,2.0,...,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,6.0,3.0,7.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1.3,1.3,4.9,8.0,5.5,8.0,3.7,6.0,8.1,9.2,...,3.3,1.4,1.3,1.3,1.3,9.5,3.8,9.7,5.7,8.9
2019-12-28,9.3,10.2,2.9,6.8,4.5,6.1,6.3,-0.1,0.2,-0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
def open_hdf4_files(file_list: list, dust_score_pm_matches: list):
    """
    Open list of AIRS HDF4 files and read dust_score, latitude, longitude and run 
    other functions to work with them.
    
    Parameters:
        file_list (list): List of file paths to HDF4 files.
    """
    start_time=time.time()
    for file in file_list:
        file_path=os.path.join(file_ext,file)
        print("Opening file:", file_path)
        try:
            hdf_file = SD(file_path)
            # Print information about the HDF4 file
            print("File Info:")
            print(hdf_file.info())
            # Select dust_score, lat and long     
            dust_score=hdf_file.select('dust_score')[:]
            long=hdf_file.select('Longitude')[:]
            lat=hdf_file.select('Latitude')[:]
            coords=coordinates(lat,long)
            # Find timestamp of production
            global_attributes = hdf_file.attributes()
            # Get raw productiondatetime from hdf file
            datetime_hdf_raw=find_rangedatetime(global_attributes)
            print(datetime_hdf_raw)
            # Convert raw string into datetime object
            datetime_converted=datetime.strptime(datetime_hdf_raw,"%Y-%m-%dT%H:%M:%S.%fZ")
            print(datetime_converted)
            
            # Round time from converted hdf datetime object to nearest hour 
            datetime_rounded_time=round_nearest_hour(datetime_converted)
            print("UTC: ",datetime_rounded_time)
            #Convert UTC to MST (only valid for CAMS49)
            datetime_rounded_time_mst=datetime_rounded_time-timedelta(hours=7)
            print("MST: ",datetime_rounded_time_mst)
            '''
            Do the same for CAMS1028 station
            '''
            # Convert datetime object (date part only) into string
            datetime_converted_str_mst=datetime_rounded_time_mst.date().strftime("%Y-%m-%d")

            # Use the date string to find the matching date row in the TECQ file
            tecq_date_found=tecq_cams49.loc[datetime_converted_str_mst]
            print(tecq_date_found)



            # Make an array of an hour before and after the flyby's rounded hour
            datetime_range_mst=[(datetime_rounded_time_mst-timedelta(hours=1)).time(), (datetime_rounded_time_mst.time()), (datetime_rounded_time_mst+timedelta(hours=1)).time()]
            
            # Find the mean PM concentration at time of satellite flyby at TECQ station
            tecq_pm_hourmean_cams49=np.mean(tecq_date_found[datetime_range_mst])
            tecq_pm_daymax_cams49=np.max(tecq_date_found)
            print("Mean 3hour PM CAMS49: ", tecq_pm_hourmean_cams49)
            print("Max Daily PM CAMS49: ", tecq_pm_daymax_cams49)

            # Find the dust_score mean at TECQ station's location
            array_to_match=[CAMS49[0],CAMS49[1]]
            matched_coords=match_coords(coords, array_to_match)
            mask=create_mask(matched_coords)
            #print(mask)
            dust_score_masked=dust_score[mask]
            #print(dust_score_masked)
            coords_masked=coords[mask]
            dust_score_mean=np.mean(dust_score_masked)
            dust_score_max=np.max(dust_score_masked)
            print("Mean dust_score: ",dust_score_mean)
            print("Max dust_score: ",dust_score_max)
            if dust_score_max>380:
                print("Found dust_score above 380, printing plot...")
                draw_grid_plot(coords, coords_masked, file, datetime_hdf_raw, dust_score)
            dust_score_pm_tuple=[tecq_pm_hourmean_cams49, dust_score_mean]
            dust_score_pm_matches.append(dust_score_pm_tuple)

            dust_score_pm_matches_max.append([tecq_pm_daymax_cams49, dust_score_max])
            
            #print(dust_score_pm_matches) 

            
            print("\n")
            # Close the HDF4 file
            #hdf_file.end()
        except Exception as e:
            print("Error opening file:", file_path)
            print(e)
            print("\n")
    end_time=time.time()
    runtime = end_time - start_time
    print("Execution time:", runtime, "sec")
    return dust_score_pm_matches     
if __name__ == "__main__":
    # Array of matched PM to dust_score
    dust_score_pm_matches=[['PM', 'dust_score']]
    dust_score_pm_matches_max=[['PM', 'dust_score']]
    # Open HDF4 files
    #dust_score_pm_matches=open_hdf4_files(file_list,dust_score_pm_matches)
    

In [94]:
'''
import matplotlib.pyplot as plt

dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches_max[1:])
#dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches[1:])

mask_matches=np.isnan(dust_score_pm_matches_nohdr).any(axis=1)
dust_score_pm_matches_nohdr_nonan=dust_score_pm_matches_nohdr[~mask_matches]
print(len(dust_score_pm_matches_max))

dust_score_axis=np.array(dust_score_pm_matches_nohdr_nonan)
x_pm=dust_score_axis[:,0]
y_dust_score=dust_score_axis[:,1]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

ax.scatter(x_pm,y_dust_score)

#ax.plot(CAMS49[1],CAMS49[0], 'ro', markersize=3, transform=ccrs.PlateCarree())

# Add title and show the plot
plt.title('Mean PM2.5 vs Dust_score')
plt.xlabel("PM2.5 (µg/m3)")
plt.ylabel("Dust_score")
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\ndust_score_pm_matches_nohdr=np.array(dust_score_pm_matches_max[1:])\n#dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches[1:])\n\nmask_matches=np.isnan(dust_score_pm_matches_nohdr).any(axis=1)\ndust_score_pm_matches_nohdr_nonan=dust_score_pm_matches_nohdr[~mask_matches]\nprint(len(dust_score_pm_matches_max))\n\ndust_score_axis=np.array(dust_score_pm_matches_nohdr_nonan)\nx_pm=dust_score_axis[:,0]\ny_dust_score=dust_score_axis[:,1]\n\nfig = plt.figure(figsize=(6, 6))\nax = fig.add_subplot(1, 1, 1)\n\nax.scatter(x_pm,y_dust_score)\n\n#ax.plot(CAMS49[1],CAMS49[0], \'ro\', markersize=3, transform=ccrs.PlateCarree())\n\n# Add title and show the plot\nplt.title(\'Mean PM2.5 vs Dust_score\')\nplt.xlabel("PM2.5 (µg/m3)")\nplt.ylabel("Dust_score")\nplt.show()\n'

In [79]:
corr_coefficient, p_value = pearsonr(x_pm, y_dust_score)

print("Pearson correlation coefficient:", corr_coefficient)
print("p-value:", p_value)

Pearson correlation coefficient: 0.03303929448535588
p-value: 0.3527952816414486


In [93]:
'''
plt.hist(x_pm, alpha=0.5, label='PM2.5')
plt.hist(y_dust_score, alpha=0.5, label='Dust_score')

plt.xlabel('PM2.5 & Dust_score Values')
plt.ylabel('Frequency')
plt.title('Max PM2.5 & Dust_score Data Distribution')

plt.legend()
'''

"\nplt.hist(x_pm, alpha=0.5, label='PM2.5')\nplt.hist(y_dust_score, alpha=0.5, label='Dust_score')\n\nplt.xlabel('PM2.5 & Dust_score Values')\nplt.ylabel('Frequency')\nplt.title('Max PM2.5 & Dust_score Data Distribution')\n\nplt.legend()\n"

In [90]:
#ax.set_extent([-115, -85, 15, 40], crs=ccrs.PlateCarree())  # Adjust to focus on a specific region
